In [ ]:
#######################################################
#######################################################
############    COPYRIGHT - DATA SOCIETY   ############
#######################################################
#######################################################

## OUTLIER DETECTION DAY 2/OUTLIER DETECTION PART 3 ##

## NOTE: To run individual pieces of code, select the line of code and
##       press ctrl + enter for PCs or command + enter for Macs




In [ ]:
#=================================================-
#### Slide 26: Loading packages  ####

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest




In [ ]:
#=================================================-
#### Slide 27: Directory settings  ####

# Set 'main_dir' to location of the project folder
from pathlib import Path 
home_dir = Path(".").resolve()
main_dir = home_dir.parent
print(main_dir)
data_dir = str(main_dir) + "/data"
print(data_dir)




In [ ]:
#=================================================-
#### Slide 28: Load the dataset  ####

paysim = pd.read_csv(str(data_dir)+"/paysim_transactions.csv")
paysim.head()




In [ ]:
#=================================================-
#### Slide 29: Prepare the dataset for modeling  ####

# Drop columns. 
paysim = paysim.drop(['step', 'type','nameOrig', 'nameDest', 'isFlaggedFraud'], axis = 1)
paysim.columns




In [ ]:
#================================================= -
#### Slide 30: LOF on fraud dataset  ####

# Stratified split
X = paysim[['oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']]
y = paysim['isFraud']
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=1)

# Isolate non‑fraud for LOF training
X_train = X_train_full[y_train_full == 0]

# 3) <b>Scale data</b> (Euclidean distance requires scaling)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)




In [ ]:
#================================================= -
#### Slide 31: What does `novelty=True` do?  ####

# Unsupervised mode (no novelty) — predict on new data will error:
lof_unsup = LocalOutlierFactor(n_neighbors=5, novelty=False)
# lof_unsup.predict(X_test_scaled)  # ← Not allowed

# Novelty mode — fit on inliers, then predict on new samples:
lof_sup = LocalOutlierFactor(n_neighbors=5, contamination=0.1, novelty=True)
lof_sup.fit(X_train_scaled)
_ = lof_sup.predict(X_test_scaled)  # Valid




In [ ]:
#=================================================-
#### Slide 32: Create and fit LOF model  ####

lof = LocalOutlierFactor(
    n_neighbors=5,
    contamination=0.1,
    novelty=True
)
lof.fit(X_train_scaled)




In [ ]:
#=================================================-
#### Slide 33: Test predictions  ####

y_pred = lof.predict(X_test_scaled)

# Map: -1→1 (fraud), +1→0 (normal)
y_pred = np.where(y_pred == -1, 1, 0)




In [ ]:
#================================================= -
#### Slide 34: Find TPR and TNR  ####

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
non_fraud_eval = tn / (tn + fp)
print(non_fraud_eval)
fraud_eval = tp / (tp + fn)
print(fraud_eval)




In [ ]:
#=================================================-
#### Slide 35: Load performance_df dataframe  ####

performance_df = pickle.load(open(str(data_dir) + "/performance_anomalies.sav", "rb"))  
# append the new row for LOF
s = pd.Series(
    ['LOF', fraud_eval, non_fraud_eval],
    index=['model_name', 'TPR', 'TNR']
)

performance_df = pd.concat([performance_df, s.to_frame().T], ignore_index=True)
performance_df




In [ ]:
#=================================================-
#### Slide 36: Exercise 1  ####






In [ ]:
#================================================= -
#### Slide 40: LOF model - tuning neighborhood size  ####

results = []
for n in range(3, 21):
    lof_n = LocalOutlierFactor(n_neighbors=n, contamination=0.1, novelty=True)
    lof_n.fit(X_train_scaled)
    pred_n = np.where(lof_n.predict(X_test_scaled) == -1, 1, 0)
    tn, fp, fn, tp = confusion_matrix(y_test, pred_n).ravel()
    results.append((n, tp/(tp+fn), tn/(tn+fp)))

lof_df = pd.DataFrame(results, columns=['n_neighbors','TPR','TNR'])




In [ ]:
#================================================= -
#### Slide 41: LOF model - tuning neighborhood size  ####

plt.figure(figsize=(5,5))
plt.plot(lof_df['n_neighbors'], lof_df['TNR'], label='TNR')
plt.plot(lof_df['n_neighbors'], lof_df['TPR'], label='TPR')
plt.xlabel('Neighbor')
plt.ylabel('Score')
plt.legend(loc="lower right")
plt.show()




In [ ]:
#================================================= -
#### Slide 42: LOF model - tuning contamination  ####

cont_vals = [0.01,0.03,0.05,0.1,0.2,0.3]
results = []
for c in cont_vals:
    lof_c = LocalOutlierFactor(n_neighbors=10, contamination=c, novelty=True)
    lof_c.fit(X_train_scaled)
    pred_c = np.where(lof_c.predict(X_test_scaled)==-1, 1, 0)
    tn, fp, fn, tp = confusion_matrix(y_test, pred_c).ravel()
    results.append((c, tp/(tp+fn), tn/(tn+fp)))

cont_df = pd.DataFrame(results, columns=['contamination','TPR','TNR'])




In [ ]:
#================================================= -
#### Slide 43: LOF model - tuning contamination  ####

plt.plot(cont_df['contamination'], cont_df['TPR'], label='TPR')
plt.plot(cont_df['contamination'], cont_df['TNR'], label='TNR')
plt.xlabel('contamination')
plt.legend(loc='lower right')
plt.show()




In [ ]:
#================================================= -
#### Slide 44: Optimized LOF model  ####

lof_opt = LocalOutlierFactor(
    n_neighbors=10, contamination=0.1, novelty=True
)
lof_opt.fit(X_train_scaled)




In [ ]:
#=================================================-
#### Slide 45: Test predictions  ####

y_opt = np.where(lof_opt.predict(X_test_scaled)==-1, 1, 0)
tn, fp, fn, tp = confusion_matrix(y_test, y_opt).ravel()
non_fraud_eval = tn / (tn + fp)
print(non_fraud_eval)
fraud_eval = tp / (tp + fn)
print(fraud_eval)




In [ ]:
#=================================================-
#### Slide 46: Add scores to the performance dataframe  ####

# append the new row for LOF
s2 = pd.Series(
    ['LOF_optimised', fraud_eval, non_fraud_eval],
    index=['model_name', 'TPR', 'TNR']
)

performance_df = pd.concat([performance_df, s2.to_frame().T], ignore_index=True)
performance_df




In [ ]:
#=================================================-
#### Slide 48: Exercise 2  ####






In [ ]:
#=================================================-
#### Slide 50: Save results as a pickle  ####

pickle.dump(performance_df, open(str(data_dir) + "/performance_anomalies.sav","wb"))
pickle.dump(X_train_scaled, open(str(data_dir) + "/X_train_scaled.sav","wb"))
pickle.dump(X_test_scaled, open(str(data_dir) + "/X_test_scaled.sav","wb"))
pickle.dump(y_test, open(str(data_dir) + "/y_test.sav","wb"))


#######################################################
####  CONGRATULATIONS ON COMPLETING THIS MODULE!   ####
#######################################################


In [ ]:
#######################################################
#######################################################
############    COPYRIGHT - DATA SOCIETY   ############
#######################################################
#######################################################

## OUTLIER DETECTION DAY 2/OUTLIER DETECTION PART 4 ##

## NOTE: To run individual pieces of code, select the line of code and
##       press ctrl + enter for PCs or command + enter for Macs




In [ ]:
#=================================================-
#### Slide 2: Loading packages  ####

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

from sklearn.preprocessing import StandardScaler  
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest




In [ ]:
#=================================================-
#### Slide 3: Directory settings  ####

# Set 'main_dir' to location of the project folder
from pathlib import Path 
home_dir = Path(".").resolve()
main_dir = home_dir.parent
print(main_dir)
data_dir = str(main_dir) + "/data"
print(data_dir)




In [ ]:
#=================================================-
#### Slide 4: Load pickled data from previous module  ####

performance_df = pickle.load(open(str(data_dir)+"/performance_anomalies.sav","rb"))  
X_train_scaled = pickle.load(open(str(data_dir)+"/X_train_scaled.sav","rb")) 
X_test_scaled = pickle.load(open(str(data_dir)+"/X_test_scaled.sav","rb")) 
y_test = pickle.load(open(str(data_dir)+"/y_test.sav","rb")) 




In [ ]:
#=================================================-
#### Slide 5: Data: load energy consumption  ####

pjm_energy = pd.read_csv(str(data_dir)+"/PJME_hourly.csv")
pjm_energy.head()




In [ ]:
#=================================================-
#### Slide 6: Data: preprocessing  ####

pjm_energy['Datetime'] = pd.to_datetime(pjm_energy['Datetime'])
pjm_energy.info()
pjm_energy = pjm_energy[pjm_energy['Datetime'] > '2018-01-01 00:00:00']
pjm_energy.shape




In [ ]:
#=================================================-
#### Slide 7: Visualize the data: line plot  ####

pjm_energy.plot(x='Datetime', y='PJME_MW', figsize=(17,6))
plt.xlabel('Date time')
plt.ylabel('Energy Consumption')
plt.title('Energy consumption (MW) at each hour in 2018')
plt.show()




In [ ]:
#=================================================-
#### Slide 9: Create and fit LOF model: energy consumption  ####

lof_energy_model = LocalOutlierFactor(n_neighbors = 50, 
                                      contamination = 0.01, 
                                      novelty = False)
                                      
pjm_energy['anomaly'] = lof_energy_model.fit_predict(pd.DataFrame(pjm_energy['PJME_MW']))




In [ ]:
#=================================================-
#### Slide 10: LOF - visualize anomalies  ####

# visualization
fig, ax = plt.subplots(figsize=(17,6))

a = pjm_energy.loc[pjm_energy['anomaly'] == -1, ['Datetime', 'PJME_MW']] #anomaly

ax.plot(pjm_energy['Datetime'], pjm_energy['PJME_MW'], color='blue', label = 'Normal')
ax.scatter(a['Datetime'],a['PJME_MW'], color='red', label = 'Anomaly')
plt.legend()
plt.show();




In [ ]:
#=================================================-
#### Slide 11: LOF - visualize anomalies  ####

lower_threshold = pjm_energy['PJME_MW'].quantile(0.25)
upper_threshold = pjm_energy['PJME_MW'].quantile(0.75)
lof_anomalies = pjm_energy[pjm_energy['anomaly'] == -1]
# Upper range of anomalies
lof_upper_anomalies = lof_anomalies[lof_anomalies['PJME_MW'] > upper_threshold]['PJME_MW']

# Lower range of anomalies
lof_lower_anomalies = lof_anomalies[lof_anomalies['PJME_MW'] < lower_threshold]['PJME_MW']




In [ ]:
#=================================================-
#### Slide 12: LOF - visualize anomalies  ####

plt.rcParams.update({'font.size': 20})
plt.hist(lof_lower_anomalies)
plt.xlabel("Anomaly points")
plt.ylabel("Frequency")
plt.title("Lower range of anomalies")
plt.show()
plt.hist(lof_upper_anomalies)
plt.xlabel("Anomaly points")
plt.ylabel("Frequency")
plt.title("Upper range of anomalies")
plt.show()




In [ ]:
#=================================================-
#### Slide 13: Exercise 3  ####






In [ ]:
#=================================================-
#### Slide 24: Create and fit isolation forest model  ####

iforest = IsolationForest(n_estimators=100, contamination = 0.1)

# model fitting
iforest.fit(X_train_scaled)




In [ ]:
#=================================================-
#### Slide 25: Test predictions  ####

fraud_pred = iforest.predict(X_test_scaled)
fraud_pred
fraud_pred[fraud_pred == 1] = 0
fraud_pred[fraud_pred == -1] = 1




In [ ]:
#================================================= -
#### Slide 26: Find TPR and TNR  ####

tn, fp, fn, tp = confusion_matrix(y_test, fraud_pred).ravel()
non_fraud_eval = tn / (tn + fp)
print(non_fraud_eval)
fraud_eval = tp / (tp + fn)
print(fraud_eval)




In [ ]:
#=================================================-
#### Slide 27: Load performance_df dataframe  ####

s3 = pd.Series(
    ['Isolation Forest', fraud_eval, non_fraud_eval],
    index=['model_name', 'TPR', 'TNR']
)

performance_df = pd.concat([performance_df, s3.to_frame().T], ignore_index=True)
performance_df




In [ ]:
#=================================================-
#### Slide 30: Exercise 4  ####






In [ ]:
#=================================================-
#### Slide 32: Isolation forest on time series data  ####

isolation_energy = IsolationForest(n_estimators=100, contamination = 0.01)

# model fitting
isolation_energy.fit(pd.DataFrame(pjm_energy['PJME_MW']))
pjm_energy['anomaly'] = isolation_energy.predict(pd.DataFrame(pjm_energy['PJME_MW']))




In [ ]:
#=================================================-
#### Slide 33: Isolation forest - visualize anomalies  ####


# visualization
fig, ax = plt.subplots(figsize=(15,5))

a = pjm_energy.loc[pjm_energy['anomaly'] == -1, ['Datetime', 'PJME_MW']] #anomaly

ax.plot(pjm_energy['Datetime'], pjm_energy['PJME_MW'], color='blue', label = 'Normal')
ax.scatter(a['Datetime'],a['PJME_MW'], color='red', label = 'Anomaly')
plt.legend()
plt.show()




In [ ]:
#=================================================-
#### Slide 34: Isolation forest - visualize anomalies  ####

# visualization
lower_threshold = pjm_energy['PJME_MW'].quantile(0.25)
upper_threshold = pjm_energy['PJME_MW'].quantile(0.75)
if_anomalies = pjm_energy[pjm_energy['anomaly'] == -1]

if_upper_anomalies = if_anomalies[if_anomalies['PJME_MW'] > upper_threshold]['PJME_MW']
if_lower_anomalies = if_anomalies[if_anomalies['PJME_MW'] < lower_threshold]['PJME_MW']




In [ ]:
#=================================================-
#### Slide 35: Isolation forest - visualize anomalies  ####

plt.hist(if_lower_anomalies)
plt.xlabel("Anomaly points")
plt.ylabel("Frequency")
plt.title("Lower range of anomalies")
plt.show()
plt.hist(if_upper_anomalies)
plt.xlabel("Anomaly points")
plt.ylabel("Frequency")
plt.title("Upper range of anomalies")
plt.show()




In [ ]:
#=================================================-
#### Slide 36: Compare anomalies  ####

plt.hist(lof_upper_anomalies,alpha = 0.5,label='LOF anomalies')
plt.hist(if_upper_anomalies,alpha = 0.5,label='IF anomalies')
plt.title('Upper range of anomalies')
plt.legend(loc='upper right')
plt.show()
plt.hist(lof_lower_anomalies,alpha = 0.5,label='LOF anomalies')
plt.hist(if_lower_anomalies,alpha = 0.5,label='IF anomalies')
plt.title('Lower range of anomalies')
plt.legend(loc='upper right')
plt.show()




In [ ]:
#=================================================-
#### Slide 37: Exercise 5  ####




#######################################################
####  CONGRATULATIONS ON COMPLETING THIS MODULE!   ####
#######################################################
